####**Imports**

In [29]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

## **Data**

We will be using the https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html dataset to test the model.

In [30]:
# Load the California Housing dataset
data = fetch_california_housing()

In [31]:
# Split the dataset into features (X) and target variable (y)
X = data.data
y = data.target

In [32]:
scaler = StandardScaler()

# Fit the scaler to the features
scaler.fit(X)

# Perform feature scaling on the features
X_scaled = scaler.transform(X)

In [33]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Print the shapes of the resulting sets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (16512, 8)
y_train shape: (16512,)
X_test shape: (4128, 8)
y_test shape: (4128,)


## **Linear Regression Implementation**

Here an implementation of Linear Regression(from scratch) using only numpy is provided. The LinearRegression class allows to fit the model using the OLS method or Gradient Descent.

In [34]:
class LinearRegression_scratch():
  """
  Linear Regression.

  Implements Linear Regression bases on OLS or Gradient Descent, as chosen by the user.

  """

  def __init__(self,learning_rate=0.01,num_iterations=1000,lambda_val=0.01,regularization=False,verbose=False):
    """
    __init__

    Initializes the coefficients and intercept to None.

    """
    self.coefficients = None
    self.intercept = None
    self.learning_rate = learning_rate
    self.num_iterations = num_iterations
    self.lambda_val = lambda_val
    self.regularization = False
    self.verbose = verbose

  def fit(self,X,y,method="ols",learning_rate = None,num_iterations = None,lambda_val = None,verbose=None):
    """
    Fit.

    It fits the coefficientes w = (w1,...,wn) to the data in order to minimize the residual sum of squares between the data and the linear model.
    Two approaches are supported: "ols" or "gradientdesc"

    Parameters
    ----------
    X : numpy.ndarray. Shape (n_samples,n_features)
        Training data.

    y : numpy.ndarray. Shape (n_samples,)
        Target training data.

    method : string.
             The method used to estimate the coefficients of the linear model.

    learing_rate : float.
                   The learning rate used to modify the parameters.

    num_iterations : int.
                     Number of iterations for the application of gradient descent.

    lambda_val : float.
                 Lambda coefficient for applying regularization.

    verbose: bool.
             Wether to display iteration information during training.

    Returns
    -------
    self : object
           Fitted model

    """
    if verbose is not None:
      self.verbose=verbose
    if (method == "ols"):
      self.fit_ols(X,y)
    elif (method =="gradientdesc"):
      self.fit_sgd(X,y,learning_rate=learning_rate,num_iterations=num_iterations,lambda_val=lambda_val)
    else:
      raise ValueError("Invalid fitting method. Please choose 'ols' or 'gradientdesc'.")

  def fit_ols(self,X,y):
    # Add a column of ones for the intercept term
    ones = np.ones((X.shape[0],1))
    X = np.hstack((ones, X))

    # Calculate coefficients using the Ordinary Least Squares formula: B = X'X^(-1)X'y
    self.coefficients = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),y)

  def fit_sgd(self,X,y,learning_rate,num_iterations,lambda_val):

    if lambda_val is not None:
      self.regularization = True
      self.lambda_val = lambda_val

    # Get the number of samples and features
    n_samples, n_features = X.shape

    # Add a column of ones for the intercept term
    ones = np.ones((n_samples,1))
    X = np.hstack((ones, X))

    # Initialize the coefficients to zeros
    self.intercept = 0.0
    self.coefficients = np.ones(n_features+1)

    # For each iteration, generate predictions, calculate the error and update the coefficients according to the learning rate and the gradient of the loss.
    for iter in range(num_iterations):

      # Generate predictions
      y_pred = np.dot(X,self.coefficients)

      # Calculate the error
      error = y_pred - y

      # Update intercept and coefficients
      if not self.regularization :
        self.coefficients = self.coefficients - (learning_rate * (1/n_samples) * np.dot(X.T,error))
      else:
        self.coefficients = self.coefficients - (learning_rate * (1/n_samples) * np.dot(X.T,error) + (2 * (self.lambda_val /n_samples) * self.coefficients))

      if self.verbose:
        if (iter + 1) % 100 == 0:
          mse = self.evaluate(X[:,1:],y)
          print(f"Iteration {iter + 1}: Mean Squared Error = {mse:.2f}")

  def predict(self,X):
    """
    Predict.

    Predicts the target values with the estimated linear model.

    Parameters
    ----------
    X : numpy.ndarray. Shape (n_samples,n_features)
        Data to generate the prediction on.

    Returns
    -------
    y_pred : numpy.ndarray. Shape (n_samples,1)
             Generated predictions

    """
    # Add a column of ones for the intercept term
    ones = np.ones((X.shape[0],1))
    X = np.hstack((ones, X))

    # Generate predictions with y_pred = Bx
    y_pred = np.dot(X,self.coefficients)

    return y_pred

  def get_coefficients(self):
    """
    Get coefficients.

    Returns the estimated coefficients.

    Returns
    -------
    coefficients: numpy.ndarray. Shape(num_features + 1,1)
                  Estimated coefficients and intercept.
    """
    coefficients = np.concatenate(([self.intercept],self.coefficients))
    return coefficients

  def evaluate(self,X,y):
    """
    Evaluate.

    Evaluate the predictions generated by the fitted model.

    Parameters
    ----------
    X : numpy.ndarray. Shape (n_samples,n_features)
        Test data.

    y : numpy.ndarray. Shape (n_samples,)
        Target test data.

    Returns
    -------
    metric : float.
             Calculated metric.
    """
    # Generate predictions on test data
    pred = self.predict(X)
    mse = np.mean((y - pred)**2)

    return mse


In [35]:
# Insantiate models
lr_sklearn = LinearRegression()
lr_ols = LinearRegression_scratch()
lr_sgd = LinearRegression_scratch()

# Fit models
lr_sklearn.fit(X_train,y_train)
lr_ols.fit(X_train,y_train,method='ols')
lr_sgd.fit(X_train,y_train,method="gradientdesc",learning_rate=0.01, num_iterations = 5000,lambda_val=0.01,verbose=False)

# Evaluate models
mse_sklearn = lr_sklearn.score(X_test,y_test)
mse_ols = lr_ols.evaluate(X_test,y_test)
mse_sgd = lr_sgd.evaluate(X_test,y_test)

print(f"MSE for OLS(Sklearn): {mse_sklearn}")
print(f"MSE for OLS: {mse_ols}")
print(f"MSE for SGD: {mse_sgd}")

MSE for OLS(Sklearn): 0.5757877060324511
MSE for OLS: 0.5558915986952444
MSE for SGD: 0.5600119427714327
